<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/pl_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hhdwj01t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hhdwj01t
  Resolved https://github.com/huggingface/transformers to commit 25b0f2033ba23e354ef2f665764248fcbb3f49ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8001073 sha256=69f42304438c10396f4fde6a374392943f3bf37f7a19e9c904ff6a38dac09b36
  Stored in directory: /tmp/pip-ephem-wheel-cache-xrphr8id/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "pl", split="validation")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=a67aa0e92dd299f9bf07241b8ccd032945ca2203bfc0c57d0bb84e7b07708359
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [6]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 63.4MiB/s]


In [7]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [8]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="pl")
    return result["text"]

In [9]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [10]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [11]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.38235294117647056, Precision: 0.6666666666666666, Recall: 0.7272727272727273, F1: 0.6956521739130435
Processed 2 files, WER: 0.37142857142857144, Precision: 0.6774193548387096, Recall: 0.7, F1: 0.6885245901639343
Processed 3 files, WER: 0.13636363636363635, Precision: 0.9090909090909091, Recall: 0.9090909090909091, F1: 0.9090909090909091
Processed 4 files, WER: 0.2222222222222222, Precision: 0.7777777777777778, Recall: 0.7777777777777778, F1: 0.7777777777777778
Processed 5 files, WER: 0.25, Precision: 0.75, Recall: 0.75, F1: 0.75
Processed 6 files, WER: 0.4, Precision: 0.5714285714285714, Recall: 0.6153846153846154, F1: 0.5925925925925927
Processed 7 files, WER: 0.6666666666666666, Precision: 0.5, Recall: 0.5, F1: 0.5
Processed 8 files, WER: 0.25, Precision: 1.0, Recall: 0.75, F1: 0.8571428571428571
Processed 9 files, WER: 0.9333333333333333, Precision: 0.35714285714285715, Recall: 0.3333333333333333, F1: 0.3448275862068965
Processed 10 files, WER: 0.333333333